# Trabalho Prático 2  #
#### André Freitas PG54707 ####
#### Bruna Macieira PG54467 ####

## Exercício 3

O algoritmo de Boneh e Franklin (BF) discutido no +Capítulo 5b:  Curvas Elípticas e sua Aritmética é uma tecnica fundamental na chamada “Criptografia Orientada à Identidade”. Seguindo as orientações definidas nesse texto, pretende-se construir usando Sagemath uma classe Python que implemente este criptosistema.

In [34]:
%pip install pycrypto
%pip install sagemath-standard
import math
from sage.all import *
from Crypto.Hash import SHA512
from sage.all import EllipticCurve, ZZ, GF, EllipticCurve

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [35]:
class BonehFranklinIBE:
    def Hash(hash_input):
        hash = SHA512.new()
        str_val = str(hash_input)
        byte_val = str_val.encode()
        hash.update(byte_val)
        hexadecimal = hash.hexdigest()
        return int(hexadecimal, 16)
    
    #Fast modular exponentiation algorithm (needed for F_{p^2})
    def fastPowerSmall(g,A):
        a = g
        b = 1
        if g == 0 and A == 0:
            return "Undefined"
        else:
            while A > 0:
                if A%2 == 1:
                    b = b*a
                a = a^2
                A = A//2
                print
            return b
        
    def textToInt(w):
        n  = 0
        counter = 0
        #counter will give us the index of each character in the string
        for i in w:
            n = n + ord(i)*(256**counter)
            counter = counter + 1
        return n

    def intToText(n):
        str = ""
        while n > 0:
            a0 = n%256
            str = str + chr(a0) #chr undoes ord. ord() inputs character and outputs integer. str inputs integer between 0 and 255 and outputs character.
            n = n//256 #This is the quotient after dividing n by 256
        return str
    
    def is_prime(n):
        if n <= 1 or (n % 2 == 0 and n > 2): 
            return False
        for i in range(3, int(math.sqrt(n)) + 1, 2):
            if n % i == 0:
                return False
        return True
    
    def Xor(a, b):
        int_a = int(a)
        int_b = int(b)
        return int_a ^^ int_b
    
    def is_in_prime_field(element, prime):
        return 0 <= element < prime
    
    #This defines a rational function g(x,y) on E whose divisor is div(g) = [P] + [Q] - [P+Q] - [O]

    #A, B coefficients of E. Use A,B = E.a_invariants()[3], E.a_invariants()[4]
    #P = E([xP, yP])

    def g(P,Q,x,y,E):
        positive_infinity = math.inf
        A,B = E.a_invariants()[3], E.a_invariants()[4]
        if P == E(0) or Q == E(0):
            return "no divisor"
        xP,yP = P[0],P[1]
        xQ,yQ = Q[0],Q[1]
        #Calculate slope of line connecting P and Q
        #JUST check if equal
        if yP == -yQ and xP == xQ:
            slope = +positive_infinity #symbol for Infinity
        elif P == Q:
            slope = (3*(xP**2) + A)/(2*yP)
        else:
            slope = (yQ - yP)/(xQ - xP)
        #return the function on E
        if slope == +positive_infinity:
            return x - xP
        else:
            return (y - yP - slope*(x - xP))/(x + xP + xQ - slope**2)
        
    def MillerAlgorithm(P,m,x,y,E):
        A,B = E.a_invariants()[3], E.a_invariants()[4]
        xP,yP = P[0],P[1]
        binary = m.digits(2) #gives number in binary
        n = len(binary) #trying to find what "n" is.
        T = P
        f = 1
        for i in range(n-2,-1,-1): #Stop once i = -1, so last number is 0...range(start, stop, step)
            f = (f**2)*BonehFranklinIBE.g(T,T,x,y,E)
            T *= 2 #T = 2T
            if binary[i] == 1:
                f = f*BonehFranklinIBE.g(T,P,x,y,E)
                T += P
        return f 

    def WeilPairing(P,Q,m,E):
        A,B = E.a_invariants()[3], E.a_invariants()[4]
        S = E.random_element()
        while m*S == E(0):
            S = E.random_element() #Pick point S that is not m-torsion. This guarantees that S isn't a linear combination of P and Q.
        xS,yS = S[0],S[1]
        QplusS = Q + S
        f_P_QplusS = BonehFranklinIBE.MillerAlgorithm(P,m,QplusS[0],QplusS[1],E)
        f_P_S = BonehFranklinIBE.MillerAlgorithm(P,m,xS,yS,E)
        num = f_P_QplusS/f_P_S

        PminusS = P - S
        f_Q_PminusS = BonehFranklinIBE.MillerAlgorithm(Q,m,PminusS[0],PminusS[1],E) #This is f_Q(P-S)
        f_Q_minusS = BonehFranklinIBE.MillerAlgorithm(Q,m,xS,-yS,E) #This is f_Q(-S)
        denom = f_Q_PminusS/f_Q_minusS

        e_m = num/denom
        return e_m
    
    def ModifiedWeilPairing(P,Q,m,E):
        Fp = GF(p)
        R.<x> = Fp[]
        Fp2.<z> = Fp.extension(x^2+x+1)      #Form F_{p^2} by adjoining z = {a nontrivial cube root of 1}
        E_zeta = EllipticCurve(Fp2, [0,1])   #Define E: y^2 = x^3 + 1 over this field
        phiQ = E_zeta([z*Q[0],Q[1]])
        A,B = E_zeta.a_invariants()[3], E_zeta.a_invariants()[4]
        P_zeta = E_zeta([P[0],P[1]])
        S = E_zeta.random_element()
        while m*S == E(0):
            S = E_zeta.random_element()
        xS,yS = S[0],S[1]
        QplusS = phiQ + S
        f_P_QplusS = BonehFranklinIBE.MillerAlgorithm(P,m,QplusS[0],QplusS[1],E_zeta)
        f_P_S = BonehFranklinIBE.MillerAlgorithm(P,m,xS,yS,E_zeta)
        num = f_P_QplusS/f_P_S

        PminusS = P_zeta - S #modify
        f_Q_PminusS = BonehFranklinIBE.MillerAlgorithm(phiQ,m,PminusS[0],PminusS[1],E_zeta) #This is f_Q(P-S)
        f_Q_minusS = BonehFranklinIBE.MillerAlgorithm(phiQ,m,xS,-yS,E_zeta) #This is f_Q(-S)
        denom = f_Q_PminusS/f_Q_minusS

        e_m = num/denom
        return e_m
    
    def KeyGen(k):
        q = random_prime((2^k) - 1, True, lbound=2^(k-1)) #Generates a random k-bit prime. False means using pseudo-primality tests.
        p = q
        l = 1 #need l for `KeyExtract`
        lq = q
        while True:
            l += 1
            lq += q
            p = lq - 1
            if p%3 == 2 and (p+1)%(q^2) != 0 and is_prime(p) == True:
                break
        E = EllipticCurve(GF(p), [0,1]) #p is public b/c the elliptic curve is known
        P = E(0)
        while P == E(0):
            Q = E.random_element()
            while Q == E(0): #make sure P is not O
                Q = E.random_element()
            h = (p+1)//q #This is to make sure P has order q.
            P = h*Q #Order of P is q1
        s = ZZ.random_element(2,q-1) #s is private master key in Z_q*.
        P_pub = s*P
        params = [p, q, l, E, P, P_pub]
        return params, s 
    
    def KeyExtract(y0, params):
        p, q, l, E, P, P_pub = params
        x0 = pow((y0^2) - 1, ((2*p)-1)//3, p) #`pow` is Python's built-in function that does fast power
        Q = E([x0,y0])
        Q_ID = l*Q #l comes from BDHGenerator. It's the integer s.t. p = lq-1
        d_ID = s*Q_ID
        return Q_ID,d_ID
    
    def Encrypt(M,Q_ID,params):
        params = [p, q, l, E, P, P_pub]
        r = ZZ.random_element(2,q-1)
        U = r*P
        g_ID = BonehFranklinIBE.ModifiedWeilPairing(Q_ID,P_pub,q,E)
        g_ID_to_r = BonehFranklinIBE.fastPowerSmall(g_ID, r)

        if BonehFranklinIBE.is_in_prime_field(g_ID_to_r, p):  # assuming p is your prime
            g_ID_to_r_int = int(g_ID_to_r)
        else:
            print("g_ID_to_r is not in the prime field")
            g_ID_to_r_int = 3
            
        V = BonehFranklinIBE.Xor(M, g_ID_to_r_int)
        C = [U,V] #This is the ciphertext
        return C
    
    def Decrypt(C,d_ID,params):
        params = [p, q, l, E, P, P_pub]
        U,V = C
        weil = BonehFranklinIBE.ModifiedWeilPairing(d_ID,U,q,E)
        # Ensure weil is in the prime field
        if BonehFranklinIBE.is_in_prime_field(weil, p):  # assuming p is your prime
            M = BonehFranklinIBE.Xor(V, int(weil))
        else:
            print("weil is not in the prime field")
            # Handle the case when weil is not in the prime field
            M = 3  # or some other default value
        return M

In [36]:
#Step 1: Only need to run once
params,s = BonehFranklinIBE.KeyGen(512)
p, q, l, E, P, P_pub = params
print("p = ", p)
print("q = ", q)
print("E = ", E)
print("P = ", P)
print("s = ", s)
print("l = ", l)
print("P_pub = ", P_pub)

p =  2091154225024371474480125587441554649664775463960762497704610890558576687598178973819757593250361224257008663869824793848734044168572682805714421738238111941
q =  7415440514270820831489807047665087410158778240995611693987981881413392509213400616382119125001280937081591006630584375350120724002030790091185892688787631
E =  Elliptic Curve defined by y^2 = x^3 + 1 over Finite Field of size 2091154225024371474480125587441554649664775463960762497704610890558576687598178973819757593250361224257008663869824793848734044168572682805714421738238111941
P =  (958770581434886094705427302748121824117975313141601557253157241502106894313112993076371105796733827401088056679546746674424352990931918258506638132568911197 : 1442975958774425263877797915808521412267032785952039285208203039526891054444792858147462920792372557190446461886009890606309855285634158679520535593929059168 : 1)
s =  5881092054792590867272147161227480110731672729385621824476322371496913487819736996493552932449823695607164958441208

In [37]:
#Step 2: Hash. Only need to run once for one ID.
your_ID = "user"

y0 = BonehFranklinIBE.Hash(your_ID)
print("y0 = ", y0)

y0 =  9284027244711476868736574711199185332133782967237044148153727596027875067278862165519633211530008452951448644427085930313537133830441524639977514866580706


In [38]:
#Step 3: Only need to run once.
Q_ID,d_ID = BonehFranklinIBE.KeyExtract(y0,params)
print("Q_ID = ", Q_ID)
print("d_ID = ", d_ID)

Q_ID =  (1152088691283607245682023510996961419634013082504907162253192682692446522267055530124084530275381576032144487471084468124928204341962080133026013108739130325 : 764297765387913525405055170325018320053998568578582342452468640921717072421463830096031284348911869633472416240545781139169878742182733852329044766924635175 : 1)
d_ID =  (675605404488140443840009520048664943693770696954887235969182241311404545152386871065522880336410558183171465914131724544267968744805288624097914860720747726 : 474578770932652964106122752447608083232060946555286085665133122986918376843321159339519129968168391775625246623668966181813623040478078554380991003537561195 : 1)


In [39]:
#Step 4: Encrypt
input_message = "Teste"

M = BonehFranklinIBE.textToInt(input_message)
C = BonehFranklinIBE.Encrypt(M,Q_ID,params)
U,V = C
print("M = ", M)
print("U = ", U)
print("V = ", V)

g_ID_to_r is not in the prime field
M =  435745416532
U =  (578258044114525759457092748337729195894806434545709747508978145955999943527731498978164242143272755986667142931197621396166309133193596996934171383562836107 : 712199068366646905167786871021718488254839549611732387752484396973718011453302685257292148397212463129397011130625663174441880535601394003167101638217450246 : 1)
V =  435745416535


In [40]:
#Step 5: Decrypt
M0 = BonehFranklinIBE.Decrypt(C,d_ID,params)
output_message = BonehFranklinIBE.intToText(M0)
print("M0 = ", M0)
print("messsage = ", output_message)

weil is not in the prime field
M0 =  3
messsage =  
